<a href="https://colab.research.google.com/github/aleksandrast12/PUM_projekt_PZS/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
# y to quality

df = pd.read_csv("test.csv")

#df.drop_duplicates() - nie obowiazuje gdyz nawet przy jednym powtarzajacym sie elemencie inne cechy rozniace sie moga miec wplyw na ocene jakosci

#nie musimy usuwac kolumn
columns = ['fixed acidity',"volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol","quality"]
FeaturesToConvert = ['citric acid','residual sugar','chlorides']
for feature in FeaturesToConvert:
   df[feature] = df[feature].str.strip('-_')
for feature in FeaturesToConvert:
  df[feature] = df[feature].astype('float64')
for column in columns:
   df[column] = df[column].replace({'':np.nan})
#df.isna().sum()
df['density'] = df['density'].fillna(method='pad')

for column in  df[columns]:
  df[column]=np.abs(df[column])
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)
  IQR = Q3- Q1
  df = df.drop(df.loc[df[column] > (Q3 + 1.5 * IQR)].index)
  df = df.drop(df.loc[df[column] < (Q1- 1.5 * IQR)].index)

scaler = preprocessing.StandardScaler().fit(df)
df_scaled = scaler.transform(df[columns])






#df.info()
#df.head()